# Law of Coherence — Public Data Validation Notebook

Generated: 2025-10-17

Drop the CSV/ZIP/Excel files into `/mnt/data` and run top-to-bottom.

## 0) Setup

In [ ]:

import os, zipfile, io, json, math, re
from pathlib import Path
import numpy as np, pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from scipy.signal import savgol_filter
DATA_DIR = Path('/mnt/data'); print('DATA_DIR:', DATA_DIR)
plt.rcParams['figure.figsize']=(6,4); plt.rcParams['figure.dpi']=120


## 1) File checklist

In [ ]:

from pathlib import Path
for f in ['Fig4c_average_OAM_vs_frequency.csv','oam_vs_freq.csv',
          'Fig4b_vortex_probability_vs_frequency.csv','vortex_prob_vs_freq.csv',
          'Data_Excel_Sheet.xlsx','Exp_Vortex_Location_Data.zip',
          'rooney2010_E_vs_t.csv','gm_coherence_vs_t.csv','gm_energy_vs_t.csv']:
    p = Path('/mnt/data')/f
    print(f, '->', p.exists())


## 2) Helpers

In [ ]:

from pathlib import Path
def load_first(names):
    for n in names:
        p = Path('/mnt/data')/n
        if p.exists(): return p
    return None

def exp_decay(t, A, tau, C): return A*np.exp(-t/tau)+C
def power_law(t, A, alpha, C): return A*np.power(np.maximum(t,1e-12),-alpha)+C
def fit_curve(x,y,model='exp'):
    x=np.asarray(x).ravel(); y=np.asarray(y).ravel()
    if model=='exp':
        p0=[y.max()-y.min(), (x.max()-x.min())/3, y.min()]
        popt,pcov=curve_fit(exp_decay,x,y,p0=p0,maxfev=20000); 
        return popt,np.sqrt(np.diag(pcov))
    else:
        p0=[y.max(),1.0,y.min()]; popt,pcov=curve_fit(power_law,x,y,p0=p0,maxfev=20000); 
        return popt,np.sqrt(np.diag(pcov))

def rmse(a,b): a,b=np.asarray(a),np.asarray(b); return float(np.sqrt(np.mean((a-b)**2)))


## 3) Polariton rotating-bucket sweep

In [ ]:

import pandas as pd, numpy as np, matplotlib.pyplot as plt
oam = load_first(['Fig4c_average_OAM_vs_frequency.csv','oam_vs_freq.csv'])
prob = load_first(['Fig4b_vortex_probability_vs_frequency.csv','vortex_prob_vs_freq.csv'])
print('OAM file:',oam); print('Prob file:',prob)
fig,ax=plt.subplots()
if oam: 
    d=pd.read_csv(oam); ax.plot(d.iloc[:,0],d.iloc[:,1],'o-',label='OAM (exp)')
if prob:
    d2=pd.read_csv(prob); ax.plot(d2.iloc[:,0],d2.iloc[:,1],'s--',label='Vortex prob (exp)')
ax.set_xlabel('Rotation frequency'); ax.set_ylabel('OAM / Probability'); ax.legend(); ax.set_title('Polariton sweep'); plt.show()


## 4) Single-vortex decay (Rooney et al.) — fits

In [ ]:

decay=load_first(['rooney2010_E_vs_t.csv','vortex_decay_E_vs_t.csv'])
print('Decay file:',decay)
if decay:
    df=pd.read_csv(decay); t=df.iloc[:,0].values; E=df.iloc[:,1].values
    (Ae,taue,Ce),se=fit_curve(t,E,'exp'); (Ap,alpha,Cp),sp=fit_curve(t,E,'power')
    tt=np.linspace(t.min(),t.max(),400)
    plt.plot(t,E,'k.',label='E(t)'); plt.plot(tt,exp_decay(tt,Ae,taue,Ce),'--',label='exp')
    plt.plot(tt,power_law(tt,Ap,alpha,Cp),':',label='power')
    plt.legend(); plt.xlabel('t'); plt.ylabel('E'); plt.title('Vortex decay fits'); plt.show()
    print('Exp fit:',Ae,taue,Ce); print('Power fit:',Ap,alpha,Cp)
else:
    print('Place a CSV with columns t,E in /mnt/data')


## 5) Vortex positions (Zenodo 2548958) — counts over time

In [ ]:

from pathlib import Path
import pandas as pd, numpy as np, matplotlib.pyplot as plt, zipfile
zp=load_first(['Exp_Vortex_Location_Data.zip','vortex_positions.zip'])
print('Positions ZIP:',zp)
if zp:
    with zipfile.ZipFile(zp,'r') as zf:
        names=[n for n in zf.namelist() if n.lower().endswith(('.csv','.json'))]
        print('Found',len(names),'files; first:',names[:3])
        with zf.open(names[0]) as f:
            df=pd.read_csv(f) if names[0].lower().endswith('.csv') else pd.read_json(f)
    cols={c.lower():c for c in df.columns}
    tcol=cols.get('t') or cols.get('time') or cols.get('frame')
    nvt=df.groupby(tcol).size()
    nvt.plot(marker='o'); plt.title('Vortex count n_v(t)'); plt.xlabel('t/frame'); plt.ylabel('n_v'); plt.show()


## 6) Compare to Glyph Manifold simulation curves

In [ ]:

import numpy as np, pandas as pd, matplotlib.pyplot as plt
gmE=load_first(['gm_energy_vs_t.csv']); gmC=load_first(['gm_coherence_vs_t.csv'])
decay=load_first(['rooney2010_E_vs_t.csv','vortex_decay_E_vs_t.csv'])
if gmE and decay:
    g=pd.read_csv(gmE); e=pd.read_csv(decay)
    t=np.linspace(max(g.t.min(),e.iloc[:,0].min()), min(g.t.max(),e.iloc[:,0].max()), 300)
    Esim=np.interp(t,g.t.values,g.E.values); Eexp=np.interp(t,e.iloc[:,0].values,e.iloc[:,1].values)
    print('RMSE(E):', rmse(Esim,Eexp))
    plt.plot(t,Eexp,'k-',label='exp'); plt.plot(t,Esim,'r--',label='sim'); plt.legend(); plt.title('Energy decay: exp vs sim'); plt.show()
if gmC:
    print('Loaded gm_coherence_vs_t.csv for your own overlays in Section 3.')


## 7) Parameter fit stub (β, γ)

In [ ]:

def run_sim(beta,gamma,tgrid):
    A=1.0; tau=0.5+0.8*gamma; C=0.05+0.1*beta
    return A*np.exp(-tgrid/tau)+C

decay=load_first(['rooney2010_E_vs_t.csv','vortex_decay_E_vs_t.csv'])
if decay:
    df=pd.read_csv(decay); texp=df.iloc[:,0].values; Eexp=df.iloc[:,1].values
    tgrid=np.linspace(texp.min(),texp.max(),300)
    def objective(b,g):
        Esim=run_sim(b,g,tgrid); Einp=np.interp(tgrid,texp,Eexp); return rmse(Esim,Einp)
    grid=[(b,g,objective(b,g)) for b in np.linspace(0,1,11) for g in np.linspace(0,1,11)]
    best=min(grid,key=lambda x:x[2]); print('Best coarse (beta,gamma,RMSE)=',best)
